Problem Statement:

create this folder structure 
NamasteKart
NamasteKart->incoming_files
NamasteKart->success_files
NamasteKart->rejected_files

Read all the files from current date folder NamasteKart->incoming_files->YYYYMMDD

do the following validations for each order

1- product id should be present in product master table

2- total sales amount should be (product price from product master table * quantity)

3- the order date should not be in future

4- any field should not be empty

5- The orders should be from Mumbai or Bangalore only.



files with no issues should go to NamasteKart->success_files->YYYYMMDD folder 

if any single orders validation fail then full file should be rejected and that files should go to NamasteKart->rejected_files->YYYYMMDD folder 

for each rejected file there should be one more file created in the same folder NamasteKart->rejected_files->YYYYMMDD folder with name error_{rejected_file_name}. 
In this file only order records should be there which failed the validation and there should be one more column for each record specifying the reason of rejection
if there are more than one reason of reject for a particular order then both should be there ; separated

after processing all the files there should be an email sent to business. In the email mention below details 
total 10 incoming files , 8 successsful files and 2 rejected files for that day.
subject : validation email 2023-06-09

In [1]:
from datetime import date as dt
from datetime import datetime
import os

In [2]:
file_path='D:\\NamasteSql\\NamasteKart\\incoming_files'
rejected_file_path='D:\\NamasteSql\\NamasteKart\\rejected_files'
success_file_path='D:\\NamasteSql\\NamasteKart\\success_files'

In [3]:
filelist=os.listdir(file_path)

In [4]:
print(filelist)

['orders_1.csv', 'orders_2.csv']


In [5]:
for order in os.listdir('D:\\NamasteSql\\NamasteKart\\incoming_files'):
    print(order)

orders_1.csv
orders_2.csv


In [6]:
orders=[]
try:
    for order in os.listdir(file_path):
        path=file_path+'\\'+order
        f=open(path,mode='r')
        file_list=f.readlines()
        orders.append(file_list)
        f.close()
except:
    print("Data not loaded")

In [7]:
file_list

['order_id,order_date,product_id,quantity,sales,city\n',
 '1,2024-01-01,100,1,50000,Bangalore\n',
 '2,2024-02-01,200,2,9000,Mumbai\n',
 '3,2024-01-03,200,2,4500,Mumbai\n',
 '4,2024-01-04,500,3,30000,Bangalore\n']

In [8]:
def read_master_file():
    try:
        f=open('D:\\NamasteSql\\NamasteKart\\product_master.csv',mode='r')
        file_master=f.readlines()
        f.close() 
        return file_master
    except:
        print('Data not loaded')

In [9]:
file_master=read_master_file()

In [10]:
print(file_master)

['product_id,product_name,price,category\n', '100,iphone,50000,mobile\n', '200,chair,4500,furniture\n', '300,table,10000,furniture\n', '400,tv,25000,electronics\n', '500,fridge,10000,electronics\n']


In [11]:
product_master_id=[]
product_master_name=[]
product_master_price={}
product_master_category=[]
for i in range(1,len(file_master)):
    record_list=file_master[i].split(',')
    product_master_id.append(record_list[0])
    product_master_name.append(record_list[1])
    product_master_price[(record_list[0])]=record_list[2]
    product_master_category.append(record_list[3])
    

In [12]:
product_master_id

['100', '200', '300', '400', '500']

In [13]:
def validate_product(product_id):
    if product_id in product_master_id:
        return True
    else:
        return False

In [14]:
def validate_city(city_name):
    if(city_name=="Mumbai" or city_name=="Bangalore"):
        return True
    else:
        return False

In [15]:
def validate_order_date(order_date):
    date_object = datetime.strptime(order_date, '%Y-%m-%d').date()
    print(date_object)
    if(date_object<dt.today()):
        return True
    else:
        return False

In [16]:
def sales_amount(quantity,sales,product_id):
    return int(quantity)*int(sales)*int(product_master_price[product_id])

In [17]:
def validate_input(file_list):
    total_orders=0
    success_orders=0
    rejected_orders=0
    for j in range(1,len(file_list)):
        total_orders+=1
        order_details=file_list[j].split(",")
        order_id=order_details[0].strip()
        order_date=order_details[1].strip()
        product_id=order_details[2].strip()
        quantity=order_details[3].strip()
        sales=order_details[4].strip()
        city=order_details[5].strip()
        bool_flag=True
        verify_product_id=validate_product(product_id)
        if(not verify_product_id):
            err_msg= "invalid product id"
            bool_flag=False
        total_Sales=sales_amount(quantity,sales,product_id)
        verify_date=validate_order_date(order_date)
        if(not verify_date):
            err_msg= "invalid date"
            bool_flag=False
        verify_city=validate_city(city)
        if(not verify_city):
            err_msg= "invalid city"
            bool_flag=False
        if(order_date is None or product_id is None or quantity is None or sales is None or city is None ):
            err_msg= "invalid input"
            bool_flag=False
        if(bool_flag):
            success_orders+=1
            folder_path=success_file_path+"\\"+order_date
            file_path=success_file_path+"\\"+order_date+"\\"+str(product_id)+str(order_id)+".csv"
            if not os.path.exists(folder_path):
                try:
                    os.makedirs(folder_path,mode=0o777,exist_ok=True)   
                except PermissionError as e:
                    print(f"Permission error: {e}")
                except Exception as e:
                    print(f"An error occurred: {e}")
            with open(file_path,mode='w') as f:
                f.write(file_list[j])
        else:
            rejected_orders+=1
            folder_path=rejected_file_path+"\\"+order_date
            file_path=rejected_file_path+"\\"+order_date+"\\"+str(product_id)+str(order_id)+".csv"
            if not os.path.exists(folder_path):
                try:
                    os.makedirs(folder_path,mode=0o777,exist_ok=True)
                except PermissionError as e:
                    print(f"Permission error: {e}")
                except Exception as e:
                    print(f"An error occurred: {e}")                
            with open(file_path,mode='w') as f:
                f.write(file_list[j]+err_msg)
        subject="validation email "+order_date
        body="total "+str(total_orders)+" incoming files , "+str(success_orders)+" successsful files and "+str(rejected_orders)+" rejected files for that day."
        send_email(subject,body)
    return total_orders,success_orders,rejected_orders
            
        

In [18]:
import smtplib
from email.mime.text import MIMEText
def send_email(subject, body):
    sender = "s46634184@gmail.com"
    recipients = "xxx@gmail.com"
    password = "xxxx"
    msg = MIMEText(body)
    msg['Subject'] = subject
    msg['From'] = sender
    msg['To'] = recipients
    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp_server:
       smtp_server.login(sender, password)
       smtp_server.sendmail(sender, recipients, msg.as_string())
    print("Message sent!")

In [19]:
validate_input(file_list)

2024-01-01


SMTPAuthenticationError: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials lp17-20020a056a003d5100b006ddd182bf1csm4774712pfb.46 - gsmtp')